# SQL: Filtering Queries

## Setup

We are now installing the necessary packages to interact with the MySQL database and issue SQL queries using the notebook.

In [ ]:
# !sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic

In [ ]:
%reload_ext sql_magic

In [ ]:
from sqlalchemy import create_engine

conn_string = "mysql://{user}:{password}@{host}/?charset=utf8".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2015", encoding="utf-8"
)
engine = create_engine(conn_string)

In [ ]:
%config SQL.conn_name = 'engine'

## `WHERE`: Equality Conditions

### IMDb

In [ ]:
%%read_sql
USE imdb

#### Find the movie entry with id 64729.

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE id = 64729

#### Find the movie entry with movie title ‘Pulp Fiction’

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE name = 'Pulp Fiction'

#### Find the id of the movie “Schindler's List”. (Attention to the quote)

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE name = 'Schindler\'s List'

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE name = "Schindler's List"

#### List all the roles for the movie with id 290070. Sort them alphabetically


In [ ]:
%%read_sql
SELECT *
FROM roles
WHERE movie_id = 290070
ORDER BY role

## `WHERE`: Boolean Operators

### IMDb

In [ ]:
%%read_sql
USE imdb

#### Fetch all info for actresses (female gender) whose first name is Skyler

In [ ]:
%%read_sql
SELECT *
FROM actors
WHERE gender = 'F' AND first_name = 'Skyler'

#### Fetch all info for the director Steven Spielberg


In [ ]:
%%read_sql
SELECT *
FROM directors
WHERE first_name = 'Steven' AND last_name = 'Spielberg'

#### Fetch all info for the directors with last names Scorsese, Polanski, and Spielberg. Use the OR for your Boolean query.


In [ ]:
%%read_sql
SELECT *
FROM directors
WHERE last_name = 'Scorsese' OR last_name = 'Polanski' Or last_name = 'Spielberg'

#### Fetch all info for the directors Quentin Tarantino, Stanley Kubrick, and Orson Welles.

In [ ]:
%%read_sql
SELECT *
FROM directors
WHERE (first_name = 'Quentin' AND last_name = 'Tarantino') OR 
      (first_name = 'Stanley' AND last_name = 'Kubrick') OR 
      (first_name = 'Orson' AND last_name = 'Welles')

## `WHERE`: Inequality Queries

### IMDb

In [ ]:
%%read_sql
USE imdb

#### Find all information about movies that were released before 1895 (excl)

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE year<1895

#### Find all information about movies released between 1895 and 1898 (excl) 

Try both using Boolean operators and using the BETWEEN operator

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE year>1895 AND year<1898

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE year BETWEEN 1896 AND 1897 -- notice that BETWEEN is inclusive on both sides

#### Find all information about movies that were released before 1895 and after 2006 (inclusive)

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE year<=1895 OR year>=2006

## The `IN` operator

### IMDb

In [ ]:
%%read_sql
USE imdb

#### Fetch all info for the directors with last names Scorsese, Polanski, and Spielberg. Use `IN` for your Boolean query.

In [ ]:
%%read_sql
SELECT * 
FROM directors
WHERE last_name IN ( 'Scorsese', 'Spielberg', 'Polanski' );

#### Fetch all info for the directors Quentin Tarantino, Stanley Kubrick, and Orson Welles. Use `IN` for your Boolean query.


In [ ]:
%%read_sql
SELECT * FROM directors
WHERE (first_name, last_name) IN (
            ('Quentin', 'Tarantino'), 
            ('Stanley', 'Kubrick'), 
            ('Orson', 'Welles') 
)


## The `LIKE` operator for approximate queries

### IMDb

In [ ]:
%%read_sql
USE imdb

#### Find the entry for Alfred Hitchcock

Hint: Use an approximation for his first name


In [ ]:
%%read_sql
SELECT *
FROM directors
WHERE last_name = 'Hitchcock' AND first_name LIKE 'A%%' -- The double %% is only necessary when writing SQL 
                                                        -- within Jupyter notebooks. 

#### Find the Godfather movies, released in 1972, 1974, and 1990

In [ ]:
%%read_sql
SELECT *
FROM movies
WHERE name LIKE 'Godfather%%'  AND -- The double %% is only necessary when writing SQL 
      year IN (1972, 1974, 1990)   -- within Jupyter notebooks. 